In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [18]:
import getpass
import requests
from os import getenv
from dotenv import load_dotenv
from urllib.parse import quote
import pandas as pd
from IPython.display import display, HTML

load_dotenv(override=True)
DM_USER = getenv('DM_USER')
DM_PASS = quote(getenv('DM_PASS'))
SYS_NAMES = ["prelude", "prelude-upgrade6", "prelude-masks"] 
BASE_URL = 'dotmatics.net/browser/api'

In [38]:
DOMAIN = SYS_NAMES[2] # only this instance has this DS to extract exp ids
LIMIT = 1000
token_dct = {}
for sname in SYS_NAMES:
    token_endpoint = f'https://{sname}.{BASE_URL}/authenticate/requestToken?isid={DM_USER}&password={DM_PASS}'
    token = requests.get(token_endpoint).json()
    token_dct[sname] = token

In [20]:
exp_id_query_endpoint = f"query/{DM_USER}/100000/1422/EXPERIMENT_ID/greaterthan/1?limit={LIMIT}"
url = f"https://{DOMAIN}.{BASE_URL}/{exp_id_query_endpoint}"
headers = {"Authorization": f"Dotmatics {token_dct[DOMAIN]}"}
response = requests.get(url, headers=headers)
exp_id_list = response.json()["ids"]

In [40]:
results = []
for exp_id in reversed(list(exp_id_list[:20])):
    for sname in SYS_NAMES:
        writeup_url_endpoint = f"https://{sname}.{BASE_URL}/studies/experiment/{exp_id}/writeup/{{includeHtml}}"
        response = requests.get(writeup_url_endpoint, headers={"Authorization": f"Dotmatics {token_dct[sname]}"})
        eln_writeup = response.json()
        if type(eln_writeup) is str and "?" in eln_writeup:
            exp_summary_endpoint = f"https://{sname}.{BASE_URL}/data/{DM_USER}/100000/1423_PROTOCOL,1423_PROTOCOL_ID,1423_ISID,1423_CREATED_DATE/{exp_id}"
            response = requests.get(exp_summary_endpoint, headers={"Authorization": f"Dotmatics {token_dct[sname]}"})
            results.append({
                "Experiment ID": exp_id,
                "Writeup": eln_writeup,
                "System Name": sname,
                "Summary": response.json()
            })

In [41]:
df = pd.DataFrame(results)
df.to_csv("experiment_errors.csv", index=False)

html_output = df.to_html(index=False)
with open("experiment_errors.html", "w") as f:
    f.write(html_output)

display(HTML(html_output))

Experiment ID,Writeup,System Name,Summary
138606,"A mixture of ? (100.mg, 0.3500mmol) sodium iodide (10.49 mg, 0.07 mmol){{9:row 2}}_XXXXX_ ), sodium iodide (10.49 mg, 0.07 mmol){{9:row 2}}_XXXXX_ , N,N-diethylethanamine (0.05 mL, 0.35 mmol){{9:row 3}}_XXXXX_ and (2R,3R,4S,5S)-2-(6-aminopurin-9-yl)-5-(chloromethyl)tetrahydrofuran-3,4-diol (50.0 mg, 0.18 mmol){{9:row 4}}_XXXXX_ in IPA (? mL){{3:row 1}}_XXXXX_ was heated at 50 C over 72 hrs. No new spot was observed on TLC. Discarded3-Fluoroazetidine hydrochloride (117.14 mg, 1.05 mmol){{9:row 1}}_XXXXX_ 3-Fluoroazetidine hydrochloride (117.14 mg, 1.05 mmol){{9:row 1}}_XXXXX_",prelude,"{'status': 'error', 'message': 'failed to load project: 100000'}"
138606,"A mixture of ? (100.mg, 0.3500mmol) sodium iodide (10.49 mg, 0.07 mmol){{9:row 2}}_XXXXX_ ), sodium iodide (10.49 mg, 0.07 mmol){{9:row 2}}_XXXXX_ , N,N-diethylethanamine (0.05 mL, 0.35 mmol){{9:row 3}}_XXXXX_ and (2R,3R,4S,5S)-2-(6-aminopurin-9-yl)-5-(chloromethyl)tetrahydrofuran-3,4-diol (50.0 mg, 0.18 mmol){{9:row 4}}_XXXXX_ in IPA (? mL){{3:row 1}}_XXXXX_ was heated at 50 C over 72 hrs. No new spot was observed on TLC. Discarded3-Fluoroazetidine hydrochloride (117.14 mg, 1.05 mmol){{9:row 1}}_XXXXX_ 3-Fluoroazetidine hydrochloride (117.14 mg, 1.05 mmol){{9:row 1}}_XXXXX_",prelude-upgrade6,"{'status': 'error', 'message': 'failed to load project: 100000'}"
138606,"A mixture of ? (100.mg, 0.3500mmol) sodium iodide (10.49 mg, 0.07 mmol){{9:row 2}}_XXXXX_ ), sodium iodide (10.49 mg, 0.07 mmol){{9:row 2}}_XXXXX_ , N,N-diethylethanamine (0.05 mL, 0.35 mmol){{9:row 3}}_XXXXX_ and (2R,3R,4S,5S)-2-(6-aminopurin-9-yl)-5-(chloromethyl)tetrahydrofuran-3,4-diol (50.0 mg, 0.18 mmol){{9:row 4}}_XXXXX_ in IPA (? mL){{3:row 1}}_XXXXX_ was heated at 50 C over 72 hrs. No new spot was observed on TLC. Discarded3-Fluoroazetidine hydrochloride (117.14 mg, 1.05 mmol){{9:row 1}}_XXXXX_ 3-Fluoroazetidine hydrochloride (117.14 mg, 1.05 mmol){{9:row 1}}_XXXXX_",prelude-masks,"{'138606': {'primary': '138606', 'dataSources': {'1423': {'1': {'CREATED_DATE': '05/09/2016', 'ISID': 'h.lin', 'PROTOCOL': 'ChemELN', 'PROTOCOL_ID': '81'}}}}}"
138443,"[(22S,23R,24S,25R,26S,27R,37S,38S,39S)-39-acetoxy-24-[(2R,3S)-3-amino-2-hydroxy-3-(p-tolyl)propanoyl]oxy-22,27,38-trihydroxy-28,36,36,37-tetramethyl-30-oxo-49-oxatetracyclo[BLAH.BLAH.BLAH.BLAH.BLAH]heptadec-28-en-26-yl] benzoate (190.0 mg, 0.26 mmol){{9:row 1}}_XXXXX_ was added to DCM (200 mL){{3:row 1}}_XXXXX_ andditert-butyl carbonate (67.0 mg, 0.38 mmol){{9:row 2}}_XXXXX_ added to make [(25S,26R,27S,28R,29S,30R,41S,42S,43S)-43-acetoxy-27-[(2R,3S)-3-(tert-butoxycarbonylamino)-2-hydroxy-3-(p-tolyl)propanoyl]oxy-25,30,42-trihydroxy-31,40,40,41-tetramethyl-33-oxo-55-oxatetracyclo[BLAH.BLAH.BLAH.BLAH.BLAH]heptadec-31-en-29-yl] benzoate (100 mg, 0.12167 mmol, ?% yield){{2:row 1}}_XXXXX_",prelude,"{'status': 'error', 'message': 'failed to load project: 100000'}"
138443,"[(22S,23R,24S,25R,26S,27R,37S,38S,39S)-39-acetoxy-24-[(2R,3S)-3-amino-2-hydroxy-3-(p-tolyl)propanoyl]oxy-22,27,38-trihydroxy-28,36,36,37-tetramethyl-30-oxo-49-oxatetracyclo[BLAH.BLAH.BLAH.BLAH.BLAH]heptadec-28-en-26-yl] benzoate (190.0 mg, 0.26 mmol){{9:row 1}}_XXXXX_ was added to DCM (200 mL){{3:row 1}}_XXXXX_ andditert-butyl carbonate (67.0 mg, 0.38 mmol){{9:row 2}}_XXXXX_ added to make [(25S,26R,27S,28R,29S,30R,41S,42S,43S)-43-acetoxy-27-[(2R,3S)-3-(tert-butoxycarbonylamino)-2-hydroxy-3-(p-tolyl)propanoyl]oxy-25,30,42-trihydroxy-31,40,40,41-tetramethyl-33-oxo-55-oxatetracyclo[BLAH.BLAH.BLAH.BLAH.BLAH]heptadec-31-en-29-yl] benzoate (? mg, ? mmol, ?% yield){{2:row 1}}_XXXXX_",prelude-upgrade6,"{'status': 'error', 'message': 'failed to load project: 100000'}"
138443,"[(22S,23R,24S,25R,26S,27R,37S,38S,39S)-39-acetoxy-24-[(2R,3S)-3-amino-2-hydroxy-3-(p-tolyl)propanoyl]oxy-22,27,38-trihydroxy-28,36,36,37-tetramethyl-30-oxo-49-oxatetracyclo[BLAH.BLAH.BLAH.BLAH.BLAH]heptadec-28-en-26-yl] benzoate (190.0 mg, 0.26 mmol){{9:row 1}}_XXXXX_ was added to DCM (2